# 🏆 DataTour 2025 - Solution Complète de Credit Scoring

## 🎯 Objectif
Prédire le risque de défaut de crédit avec une approche avancée combinant :
- **Feature engineering sophistiqué** : extraction de patterns temporels, ratios financiers, agrégations
- **Modèles performants** : LightGBM, XGBoost, CatBoost avec optimisation pour données déséquilibrées
- **Validation croisée stratifiée** : 5-fold pour estimation robuste de la performance
- **Ensemble optimisé** : combinaison pondérée des modèles pour maximiser l'AUC

**Métrique d'évaluation :** ROC AUC Score

---
## 📦 0. Installation des Dépendances

Installation des bibliothèques nécessaires :
- **lightgbm, xgboost, catboost** : algorithmes de gradient boosting
- **optuna** : optimisation d'hyperparamètres (pour usage futur)
- **df-squeezer** : réduction automatique de la mémoire des DataFrames

In [14]:
!pip install lightgbm xgboost catboost optuna df-squeezer -q

---
## 📚 1. Imports et Configuration

Configuration de l'environnement avec :
- Imports des librairies essentielles
- Paramètres globaux (nombre de folds, random state)
- Suppression des warnings pour une sortie plus propre

In [15]:
import gc
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool

from df_squeezer import df_squeezer

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Configuration globale
N_FOLDS = 5              # Nombre de folds pour la validation croisée
RANDOM_STATE = 42        # Seed pour la reproductibilité
VERBOSE = 100            # Fréquence d'affichage pendant l'entraînement

print("✅ Imports terminés")

✅ Imports terminés


Configuration importante :

N_FOLDS = 5 : divise les données en 5 parties pour la validation croisée

RANDOM_STATE = 42 : assure la reproductibilité des résultats

VERBOSE = 100 : contrôle la fréquence des logs pendant l'entraînement

---
## 📂 2. Chargement des Données

Chargement des fichiers parquet contenant :
- **train.parquet** : données d'entraînement avec la variable cible `flag`
- **test.parquet** : données de test pour la soumission finale

Nous analysons également la distribution de la cible pour comprendre le déséquilibre des classes.

In [38]:
print("📂 Chargement des données...")
train = pd.read_parquet("train.parquet")
test = pd.read_parquet("test.parquet")

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"\nDistribution de la cible:")
print(train['flag'].value_counts(normalize=True))
print(f"\nTaux de défaut: {train['flag'].mean():.2%}")
print(f"Ratio de déséquilibre: {(train['flag']==0).sum() / (train['flag']==1).sum():.1f}:1")

📂 Chargement des données...
Train shape: (17659833, 62)
Test shape: (654068, 62)

Distribution de la cible:
flag
0    0.966372
1    0.033628
Name: proportion, dtype: float64

Taux de défaut: 3.36%
Ratio de déséquilibre: 28.7:1
Train shape: (17659833, 62)
Test shape: (654068, 62)

Distribution de la cible:
flag
0    0.966372
1    0.033628
Name: proportion, dtype: float64

Taux de défaut: 3.36%
Ratio de déséquilibre: 28.7:1


In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17659833 entries, 0 to 17659832
Data columns (total 62 columns):
 #   Column                         Dtype
---  ------                         -----
 0   id                             int32
 1   rn                             int8 
 2   pre_since_opened               int8 
 3   pre_since_confirmed            int8 
 4   pre_pterm                      int8 
 5   pre_fterm                      int8 
 6   pre_till_pclose                int8 
 7   pre_till_fclose                int8 
 8   pre_loans_credit_limit         int8 
 9   pre_loans_next_pay_summ        int8 
 10  pre_loans_outstanding          int8 
 11  pre_loans_total_overdue        int8 
 12  pre_loans_max_overdue_sum      int8 
 13  pre_loans_credit_cost_rate     int8 
 14  pre_loans5                     int8 
 15  pre_loans530                   int8 
 16  pre_loans3060                  int8 
 17  pre_loans6090                  int8 
 18  pre_loans90                    int8 
 19

In [40]:
train.head()

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,flag
0,1678548,3,8,7,17,16,9,1,9,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,3,2,1,0,0,0
1,2834188,11,3,2,0,7,14,8,2,5,1,0,2,13,6,16,5,4,8,1,0,1,1,1,1,2,17,0,1,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,2,4,1,0,0,0
2,811902,11,9,6,11,13,14,8,2,5,1,0,2,11,6,16,5,4,8,1,1,1,1,1,11,2,17,0,1,1,0,0,0,0,0,0,0,0,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,0,0,0
3,836450,1,16,0,13,0,4,9,5,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,0
4,1769024,15,9,9,4,8,1,11,1,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,1,1,0


In [41]:
test.head()

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,id_x_rn
0,1472943,2,10,4,16,16,13,10,10,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,1472943_x_2
1,660465,5,6,14,9,7,3,11,0,2,3,0,2,4,6,16,5,4,8,1,0,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,660465_x_5
2,1788193,3,1,0,9,9,7,2,16,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,1788193_x_3
3,2767146,3,15,8,1,16,6,13,14,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,9,5,4,0,0,0,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,3,1,0,0,2767146_x_3
4,2601698,9,19,10,4,8,1,11,4,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,9,5,4,0,0,0,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,1,1,2601698_x_9


### 🗜️ Optimisation de la mémoire

Réduction de l'empreinte mémoire avec `df_squeezer` qui :
- Convertit les types de données en types plus compacts (int64 → int8/16, float64 → float32)
- Permet de travailler avec des datasets volumineux sans saturer la RAM

In [17]:
print("\n🗜️ Optimisation mémoire...")
print("Avant:")
print(f"  Train: {train.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"  Test: {test.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

df_squeezer(train, edit=True, report=False)
df_squeezer(test, edit=True, report=False)

print("\nAprès:")
print(f"  Train: {train.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"  Test: {test.memory_usage(deep=True).sum() / 1024**2:.1f} MB")


🗜️ Optimisation mémoire...
Avant:
  Train: 1094.7 MB
  Test: 346.8 MB

Après:
  Train: 1094.7 MB
  Test: 82.3 MB

Après:
  Train: 1094.7 MB
  Test: 82.3 MB


---
## 🔍 3. Analyse Exploratoire Rapide

Vérification de la qualité des données et exploration des caractéristiques clés.

### Vérification des valeurs manquantes

In [18]:
missing = train.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    print("⚠️ Colonnes avec valeurs manquantes:")
    print(missing)
else:
    print("✅ Aucune valeur manquante détectée")

✅ Aucune valeur manquante détectée


### Analyse des séquences de paiement

Les colonnes `enc_paym_0` à `enc_paym_24` représentent l'historique des paiements sur 25 mois.  
Ce sont des features temporelles cruciales pour prédire le défaut.

In [19]:
paym_cols = [col for col in train.columns if 'enc_paym_' in col]
print(f"📊 Nombre de colonnes de paiement temporelles: {len(paym_cols)}")
print(f"Colonnes: {paym_cols[:5]} ... {paym_cols[-3:]}")
print(f"\nValeurs uniques dans enc_paym_0: {sorted(train['enc_paym_0'].unique())}")
print("\nInterprétation probable:")
print("  0 = Paiement à jour")
print("  1+ = Différents niveaux de retard/défaut")

📊 Nombre de colonnes de paiement temporelles: 25
Colonnes: ['enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4'] ... ['enc_paym_22', 'enc_paym_23', 'enc_paym_24']

Valeurs uniques dans enc_paym_0: [0, 1, 2, 3]

Interprétation probable:
  0 = Paiement à jour
  1+ = Différents niveaux de retard/défaut


---
## 🔧 4. Feature Engineering Avancé

Cette section est **cruciale** pour améliorer les performances. Nous créons des features dérivées qui capturent :
1. **Patterns temporels** : tendances, volatilité, changements de comportement
2. **Ratios financiers** : taux d'utilisation du crédit, ratios de dette
3. **Agrégations** : statistiques sur l'historique de paiement
4. **Interactions** : combinaisons de features existantes

La fonction create_advanced_features crée de nouvelles variables :

Features temporelles sur les paiements (moyennes, écarts-types, tendances)

Ratios financiers (taux d'utilisation du crédit, ratios de dette)

Features de durée et délais

Agrégations des retards de paiement


  Votre fonction create_advanced_features est impressionnante :
  - Features temporelles : moyennes, écarts-types, tendances, patterns de      
  changement
  - Ratios financiers : taux d'utilisation, ratios de dette
  - Agrégations intelligentes : score de sévérité pondéré pour les retards     
  - Encodages cycliques : pour la variable rn
  - 28 nouvelles features créées, passant de 62 à 90 colonnes

In [20]:
def create_advanced_features(df):
    """
    Crée des features avancées à partir des données brutes.
    
    Catégories de features créées :
    1. Features temporelles sur les paiements (statistiques, tendances)
    2. Ratios et relations financières
    3. Features de durée et délais
    4. Agrégations des retards de paiement
    5. Interactions entre flags binaires
    6. Encodages cycliques
    """
    df = df.copy()
    
    print("🔧 Création de features avancées...")
    initial_cols = len(df.columns)
    
    # ==================== 1. FEATURES TEMPORELLES SUR LES PAIEMENTS ====================
    paym_cols = [col for col in df.columns if 'enc_paym_' in col]
    
    if len(paym_cols) > 0:
        print("  ├─ Features temporelles sur les paiements...")
        paym_data = df[paym_cols]
        
        # Statistiques descriptives de base
        df['paym_mean'] = paym_data.mean(axis=1)
        df['paym_std'] = paym_data.std(axis=1)
        df['paym_max'] = paym_data.max(axis=1)
        df['paym_min'] = paym_data.min(axis=1)
        df['paym_median'] = paym_data.median(axis=1)
        
        # Mesures de dispersion et variabilité
        df['paym_range'] = df['paym_max'] - df['paym_min']
        df['paym_cv'] = df['paym_std'] / (df['paym_mean'] + 1e-6)  # Coefficient de variation
        
        # Détection de patterns : nombre de changements de statut
        df['paym_changes'] = (paym_data.diff(axis=1) != 0).sum(axis=1)
        
        # Comparaison récent vs ancien (6 derniers mois vs reste)
        recent_cols = paym_cols[:6] if len(paym_cols) >= 6 else paym_cols
        older_cols = paym_cols[6:] if len(paym_cols) > 6 else paym_cols
        
        df['paym_recent_mean'] = df[recent_cols].mean(axis=1)
        df['paym_older_mean'] = df[older_cols].mean(axis=1)
        df['paym_trend'] = df['paym_recent_mean'] - df['paym_older_mean']  # Amélioration ou détérioration
        
        # Comptages de valeurs spécifiques
        df['paym_count_zero'] = (paym_data == 0).sum(axis=1)  # Nombre de mois avec paiement à jour
        df['paym_count_nonzero'] = (paym_data != 0).sum(axis=1)  # Nombre de mois avec problème
        df['paym_ratio_zero'] = df['paym_count_zero'] / len(paym_cols)  # Proportion de bons paiements
        
        # Consécutivité : plus long streak de bons paiements
        df['paym_max_consecutive_zero'] = paym_data.apply(
            lambda x: max([sum(1 for _ in group) for key, group in __import__('itertools').groupby(x) if key == 0] or [0]), 
            axis=1
        )
    
    # ==================== 2. RATIOS ET RELATIONS FINANCIÈRES ====================
    print("  ├─ Ratios financiers...")
    
    # Taux d'utilisation du crédit
    if 'pre_loans_outstanding' in df.columns and 'pre_loans_credit_limit' in df.columns:
        df['utilization_rate'] = df['pre_loans_outstanding'] / (df['pre_loans_credit_limit'] + 1)
    
    # Ratio de dette en souffrance
    if 'pre_loans_total_overdue' in df.columns and 'pre_loans_outstanding' in df.columns:
        df['overdue_ratio'] = df['pre_loans_total_overdue'] / (df['pre_loans_outstanding'] + 1)
    
    # Ratio du plus gros dépassement
    if 'pre_loans_max_overdue_sum' in df.columns and 'pre_loans_credit_limit' in df.columns:
        df['max_overdue_ratio'] = df['pre_loans_max_overdue_sum'] / (df['pre_loans_credit_limit'] + 1)
    
    # ==================== 3. FEATURES DE DURÉE ====================
    print("  ├─ Features de durée et délais...")
    
    # Délai de confirmation du prêt
    if 'pre_since_opened' in df.columns and 'pre_since_confirmed' in df.columns:
        df['days_to_confirm'] = df['pre_since_opened'] - df['pre_since_confirmed']
    
    # Écart entre durée prévue et réelle
    if 'pre_pterm' in df.columns and 'pre_fterm' in df.columns:
        df['term_diff'] = df['pre_pterm'] - df['pre_fterm']
        df['term_ratio'] = df['pre_fterm'] / (df['pre_pterm'] + 1)
    
    # ==================== 4. AGRÉGATIONS DES RETARDS ====================
    print("  ├─ Agrégations des retards de paiement...")
    
    loans_cols = ['pre_loans5', 'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90']
    if all(col in df.columns for col in loans_cols):
        # Total de tous les retards
        df['total_late_payments'] = df[loans_cols].sum(axis=1)
        
        # Catégorie de retard la plus fréquente
        df['max_late_bucket'] = df[loans_cols].idxmax(axis=1).str.extract('(\\d+)')[0].astype(float)
        
        # Score de sévérité pondéré (plus le retard est long, plus le poids est élevé)
        df['late_severity_score'] = (
            df['pre_loans5'] * 1 +      # 0-5 jours
            df['pre_loans530'] * 2 +    # 5-30 jours
            df['pre_loans3060'] * 3 +   # 30-60 jours
            df['pre_loans6090'] * 4 +   # 60-90 jours
            df['pre_loans90'] * 5       # 90+ jours (très grave)
        )
    
    # ==================== 5. INTERACTIONS ENTRE FLAGS ====================
    print("  ├─ Interactions entre flags binaires...")
    
    zero_cols = [col for col in df.columns if 'is_zero_' in col]
    if len(zero_cols) > 0:
        df['count_zero_flags'] = df[zero_cols].sum(axis=1)
        df['ratio_zero_flags'] = df['count_zero_flags'] / len(zero_cols)
    
    # ==================== 6. FEATURES CYCLIQUES ====================
    print("  └─ Encodages cycliques...")
    
    # Encodage cyclique de rn (position dans la séquence)
    if 'rn' in df.columns:
        df['rn_sin'] = np.sin(2 * np.pi * df['rn'] / df['rn'].max())
        df['rn_cos'] = np.cos(2 * np.pi * df['rn'] / df['rn'].max())
    
    new_features = len(df.columns) - initial_cols
    print(f"\n✅ {new_features} nouvelles features créées")
    print(f"   Total de features: {len(df.columns)}")
    
    return df

### Application du feature engineering

On applique la fonction à nos datasets train et test.

Sépare les features (X) de la variable cible (y)

Aligne les colonnes entre train et test



In [21]:
train_fe = create_advanced_features(train)
test_fe = create_advanced_features(test)

print(f"\nShape après feature engineering:")
print(f"  Train: {train_fe.shape}")
print(f"  Test: {test_fe.shape}")

🔧 Création de features avancées...
  ├─ Features temporelles sur les paiements...
  ├─ Ratios financiers...
  ├─ Ratios financiers...
  ├─ Features de durée et délais...
  ├─ Features de durée et délais...
  ├─ Agrégations des retards de paiement...
  ├─ Agrégations des retards de paiement...
  ├─ Interactions entre flags binaires...
  ├─ Interactions entre flags binaires...
  └─ Encodages cycliques...
  └─ Encodages cycliques...

✅ 28 nouvelles features créées
   Total de features: 90
🔧 Création de features avancées...
  ├─ Features temporelles sur les paiements...

✅ 28 nouvelles features créées
   Total de features: 90
🔧 Création de features avancées...
  ├─ Features temporelles sur les paiements...
  ├─ Ratios financiers...
  ├─ Features de durée et délais...
  ├─ Agrégations des retards de paiement...
  ├─ Ratios financiers...
  ├─ Features de durée et délais...
  ├─ Agrégations des retards de paiement...
  ├─ Interactions entre flags binaires...
  └─ Encodages cycliques...

✅ 28 

---
## 📊 5. Préparation des Données

Séparation des features et de la cible, gestion des identifiants, et alignement des colonnes.

In [24]:
# Extraction de l'identifiant pour la soumission
if 'id_x_rn' in test_fe.columns:
    test_id = test_fe['id_x_rn'].values
    print("✅ Utilisation de 'id_x_rn' comme identifiant")
else:
    test_id = test_fe['id'].values
    print("✅ Utilisation de 'id' comme identifiant")

# Colonnes à exclure du train
train_exclude_cols = ['id', 'rn', 'flag']
# Colonnes à exclure du test
test_exclude_cols = ['id', 'rn']
if 'id_x_rn' in test_fe.columns:
    test_exclude_cols.append('id_x_rn')

# Séparation features/target
X = train_fe.drop(columns=train_exclude_cols)
y = train_fe['flag'].values
X_test = test_fe.drop(columns=[col for col in test_exclude_cols if col in test_fe.columns])

# Alignement des colonnes entre train et test
common_features = [col for col in X.columns if col in X_test.columns]
X = X[common_features]
X_test = X_test[common_features]

print(f"\n📋 Résumé:")
print(f"  Nombre de features finales: {len(common_features)}")
print(f"  X shape: {X.shape}")
print(f"  y shape: {y.shape}")
print(f"  X_test shape: {X_test.shape}")
print(f"  Test IDs shape: {test_id.shape}")

# Libération mémoire
del train, test, train_fe, test_fe
gc.collect()
print("\n🗑️ Mémoire libérée")

✅ Utilisation de 'id_x_rn' comme identifiant

📋 Résumé:
  Nombre de features finales: 87
  X shape: (17659833, 87)
  y shape: (17659833,)
  X_test shape: (654068, 87)
  Test IDs shape: (654068,)

🗑️ Mémoire libérée

📋 Résumé:
  Nombre de features finales: 87
  X shape: (17659833, 87)
  y shape: (17659833,)
  X_test shape: (654068, 87)
  Test IDs shape: (654068,)

🗑️ Mémoire libérée


---
## ⚙️ 6. Configuration des Modèles

Configuration des hyperparamètres pour chaque modèle avec gestion du déséquilibre via neg_pos_ratio.

Décortiquons-la :

y == 0 : identifie tous les cas de non-défaut (classe négative)

y == 1 : identifie tous les cas de défaut (classe positive)

.sum() : compte le nombre total de chaque cas

La division / nous donne le ratio

Par exemple, si nous avons :

9000 clients qui n'ont pas fait défaut (y == 0)
1000 clients qui ont fait défaut (y == 1)
Alors neg_pos_ratio = 9000/1000 = 9

Ce ratio est crucial car :

Il montre le déséquilibre dans nos données (ici 9 non-défauts pour 1 défaut)

Il est utilisé comme scale_pos_weight dans nos modèles pour :

Donner plus de poids aux cas rares (défauts)

Empêcher le modèle de toujours prédire la classe majoritaire

Équilibrer l'importance des erreurs sur chaque classe

Configure trois modèles avec leurs paramètres :

LightGBM : rapide et efficace en mémoire

XGBoost : très performant 

CatBoost : bon avec les variables catégorielles

In [34]:
# Calcul du poids des classes pour gérer le déséquilibre
neg_pos_ratio = (y == 0).sum() / (y == 1).sum()
print(f"⚖️ Ratio négatif/positif: {neg_pos_ratio:.2f}")
print(f"   Cela signifie qu'il y a ~{neg_pos_ratio:.0f} non-défauts pour 1 défaut")
print(f"   Les modèles donneront {neg_pos_ratio:.0f}x plus de poids aux cas de défaut\n")

# ==================== LIGHTGBM ====================
lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 63,            # 63 pour des arbres plus complexes
    'max_depth': 7,             #  7 pour des arbres plus profonds
    'learning_rate': 0.05,      # Pour un bon compromis vitesse/précision
    'feature_fraction': 0.7,    # Évite le surapprentissage
    'bagging_fraction': 0.7,    # Évite le surapprentissage
    'bagging_freq': 5,          # Pour la stabilité
    'n_estimators': 2000,       # Maximum d'itérations
    'early_stopping_rounds': 50, # Arrêt si pas d'amélioration
    'min_data_in_leaf': 50,    # 50 pour plus de granularité
    'max_bin': 255,            # 255 pour plus de précision dans les splits
    'scale_pos_weight': neg_pos_ratio,  # Gestion du déséquilibre
    'lambda_l1': 0.1,          # Régularisation L1
    'lambda_l2': 0.1,          # Régularisation L2
    'min_gain_to_split': 0.0,  # Splits agressifs
    'num_threads': -1          # Utilise tous les CPU
}

# ==================== XGBOOST ====================
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.05,       # Identique à LightGBM pour la cohérence
    'max_depth': 7,             # Augmenté pour correspondre à LightGBM
    'min_child_weight': 50,     # Similaire à min_data_in_leaf de LightGBM
    'subsample': 0.7,           # Identique à bagging_fraction
    'colsample_bytree': 0.7,    # Identique à feature_fraction
    'gamma': 0.0,               # Augmenté un peu pour éviter le surapprentissage
    'reg_alpha': 0.1,           # L1, identique à LightGBM
    'reg_lambda': 0.1,          # L2, identique à LightGBM
    'scale_pos_weight': neg_pos_ratio,  # Même gestion du déséquilibre
    'max_bin': 255,             # Identique à LightGBM
    'random_state': RANDOM_STATE,
    'n_estimators': 2000,       # Identique à LightGBM
    'early_stopping_rounds': 50, # Identique à LightGBM
    'n_jobs': -1,               # Utilise tous les CPU
    'tree_method': 'hist'       # Plus rapide pour gros datasets
}

# ==================== CATBOOST ====================
cat_params = {
    'loss_function': 'Logloss',  
    'eval_metric': 'AUC',
    'learning_rate': 0.05,       # Identique aux autres pour cohérence
    'depth': 7,                  # Aligné sur max_depth de LGB/XGB
    'min_data_in_leaf': 50,      # Identique à LightGBM
    'subsample': 0.7,            # Identique aux autres (bagging)
    'colsample_bylevel': 0.7,    # Identique aux autres (feature sampling)
    'l2_leaf_reg': 0.1,         # Régularisation L2, comme les autres
    'random_strength': 0.1,      # Pour introduire de la randomisation
    'scale_pos_weight': neg_pos_ratio,  # Gestion du déséquilibre
    'max_bin': 255,             # Identique aux autres
    'random_seed': RANDOM_STATE,
    'verbose': False,
    'thread_count': -1,         # Utilise tous les CPU
    'task_type': 'CPU',         # Mode CPU optimisé
    'bootstrap_type': 'Bernoulli',  # Pour activer subsample
    'boosting_type': 'Plain'    # Type de boosting standard
}

print("✅ Paramètres des 3 modèles configurés")

⚖️ Ratio négatif/positif: 28.74
   Cela signifie qu'il y a ~29 non-défauts pour 1 défaut
   Les modèles donneront 29x plus de poids aux cas de défaut

✅ Paramètres des 3 modèles configurés


---
## 🚀 7. Entraînement avec Validation Croisée Stratifiée

Dans notre code, on utilise StratifiedKFold

"Stratifié" signifie que chaque fold conserve la même proportion de classes que dans les données originales

Par exemple, si on a 10% de défauts dans les données totales, chaque fold aura aussi environ 10% de défauts

Nous utilisons une validation croisée (cross validation en anglais) stratifiée à 5 folds qui :
- Préserve la proportion de défauts dans chaque fold
- Génère des prédictions **Out-Of-Fold (OOF)** pour évaluer la performance
- Produit des prédictions test en moyennant les 5 modèles entraînés
- Utilise l'**early stopping** pour éviter le surapprentissage

La fonction train_with_cv :

Divise les données en 5 parties

Entraîne le modèle sur 4 parties et teste sur la 5ème

Répète 5 fois en changeant la partie de test

Produit des prédictions "Out-of-Fold" pour évaluer la performance

In [35]:
def train_with_cv(X, y, X_test, params, model_type='lgb', n_folds=5):
    """
    Entraîne un modèle avec validation croisée stratifiée.
    
    Paramètres:
    -----------
    X : DataFrame
        Features d'entraînement
    y : array
        Variable cible
    X_test : DataFrame
        Features de test
    params : dict
        Hyperparamètres du modèle
    model_type : str
        Type de modèle ('lgb', 'xgb', 'cat')
    n_folds : int
        Nombre de folds pour la CV
    
    Retourne:
    ---------
    oof_preds : array
        Prédictions Out-Of-Fold sur le train
    test_preds : array
        Prédictions moyennées sur le test
    oof_score : float
        Score AUC sur les prédictions OOF
    """
    kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))
    fold_scores = []
    
    print(f"\n{'='*60}")
    print(f"🚀 Entraînement {model_type.upper()} avec {n_folds} folds")
    print(f"{'='*60}\n")
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y), 1):
        print(f"\n{'─'*60}")
        print(f"📁 Fold {fold}/{n_folds}")
        print(f"{'─'*60}")
        
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        print(f"  Train size: {len(X_train):,} | Val size: {len(X_val):,}")
        
        if model_type == 'lgb':
            train_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
            
            model = lgb.train(
                params,
                train_data,
                num_boost_round=1000,
                valid_sets=[train_data, val_data],
                valid_names=['train', 'valid'],
                callbacks=[lgb.early_stopping(50), lgb.log_evaluation(VERBOSE)]
            )
            
            oof_preds[val_idx] = model.predict(X_val, num_iteration=model.best_iteration)
            test_preds += model.predict(X_test, num_iteration=model.best_iteration) / n_folds
            
        elif model_type == 'xgb':
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)
            dtest = xgb.DMatrix(X_test)
            
            model = xgb.train(
                params,
                dtrain,
                num_boost_round=1000,
                evals=[(dtrain, 'train'), (dval, 'valid')],
                early_stopping_rounds=50,
                verbose_eval=VERBOSE
            )
            
            oof_preds[val_idx] = model.predict(dval, iteration_range=(0, model.best_iteration + 1))
            test_preds += model.predict(dtest, iteration_range=(0, model.best_iteration + 1)) / n_folds
            
        elif model_type == 'cat':
            train_pool = Pool(X_train, y_train)
            val_pool = Pool(X_val, y_val)
            
            model = CatBoostClassifier(**params, iterations=1000, early_stopping_rounds=50)
            model.fit(train_pool, eval_set=val_pool, verbose=VERBOSE)
            
            oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
            test_preds += model.predict_proba(X_test)[:, 1] / n_folds
        
        fold_score = roc_auc_score(y_val, oof_preds[val_idx])
        fold_scores.append(fold_score)
        print(f"\n  ✅ Fold {fold} AUC: {fold_score:.6f}")
    
    oof_score = roc_auc_score(y, oof_preds)
    print(f"\n{'='*60}")
    print(f"📊 RÉSULTATS {model_type.upper()}:")
    print(f"{'='*60}")
    print(f"  Score OOF global:     {oof_score:.6f}")
    print(f"  Score moyen folds:    {np.mean(fold_scores):.6f}")
    print(f"  Écart-type folds:     {np.std(fold_scores):.6f}")
    print(f"  Min/Max folds:        {np.min(fold_scores):.6f} / {np.max(fold_scores):.6f}")
    print(f"{'='*60}")
    
    return oof_preds, test_preds, oof_score

### 🌟 Entraînement LightGBM

LightGBM est réputé pour sa rapidité et son efficacité mémoire.

In [ ]:
oof_lgb, test_lgb, score_lgb = train_with_cv(X, y, X_test, lgb_params, model_type='lgb', n_folds=N_FOLDS)

# Visualisation de l'importance des features
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print("\n🎯 Top 20 features les plus importantes:")
print(feature_importance.head(20))

# Visualisation graphique
plt.figure(figsize=(12, 6))
plt.barh(feature_importance.head(20)['feature'], feature_importance.head(20)['importance'])
plt.title('Top 20 Features les Plus Importantes (LightGBM)', fontsize=14, fontweight='bold')
plt.xlabel('Importance (gain)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.gca().invert_yaxis()  # Pour avoir la feature la plus importante en haut
plt.tight_layout()
plt.show()

# Statistiques sur l'importance des features
print("\n📊 Statistiques sur l'importance des features:")
print(f"  • Nombre total de features: {len(feature_importance)}")
print(f"  • Top 10 features représentent: {feature_importance.head(10)['importance'].sum() / feature_importance['importance'].sum():.1%} de l'importance totale")
print(f"  • Top 20 features représentent: {feature_importance.head(20)['importance'].sum() / feature_importance['importance'].sum():.1%} de l'importance totale")


🚀 Entraînement LGB avec 5 folds


────────────────────────────────────────────────────────────
📁 Fold 1/5
────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────
📁 Fold 1/5
────────────────────────────────────────────────────────────
  Train size: 14,127,866 | Val size: 3,531,967
  Train size: 14,127,866 | Val size: 3,531,967
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
[100]	train's auc: 0.646828	valid's auc: 0.644745
[100]	train's auc: 0.646828	valid's auc: 0.644745
[200]	train's auc: 0.65339	valid's auc: 0.649659
[200]	train's auc: 0.65339	valid's auc: 0.649659
[300]	train's auc: 0.657422	valid's auc: 0.65195
[300]	train's auc: 0.657422	valid's auc: 0.65195
[400]	train's auc: 0.660375	valid's auc: 0.653003
[400]	train's auc: 0.660375	valid's auc: 0.653003
[500]	train's auc: 0.662663	valid's auc: 0.65343
[500]	train's auc: 0.662663	valid's a

NameError: name 'model' is not defined

### 🔷 Entraînement XGBoost

XGBoost est un des algorithmes les plus performants en compétition.

In [27]:
oof_xgb, test_xgb, score_xgb = train_with_cv(X, y, X_test, xgb_params, model_type='xgb', n_folds=N_FOLDS)


🚀 Entraînement XGB avec 5 folds


────────────────────────────────────────────────────────────
📁 Fold 1/5
────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────
📁 Fold 1/5
────────────────────────────────────────────────────────────
  Train size: 14,127,866 | Val size: 3,531,967
  Train size: 14,127,866 | Val size: 3,531,967
[0]	train-auc:0.60403	valid-auc:0.60308
[0]	train-auc:0.60403	valid-auc:0.60308
[100]	train-auc:0.64830	valid-auc:0.64494
[100]	train-auc:0.64830	valid-auc:0.64494
[200]	train-auc:0.65661	valid-auc:0.65039
[200]	train-auc:0.65661	valid-auc:0.65039
[300]	train-auc:0.66166	valid-auc:0.65276
[300]	train-auc:0.66166	valid-auc:0.65276
[400]	train-auc:0.66548	valid-auc:0.65379
[400]	train-auc:0.66548	valid-auc:0.65379
[500]	train-auc:0.66874	valid-auc:0.65437
[500]	train-auc:0.66874	valid-auc:0.65437
[600]	train-auc:0.67176	valid-auc:0.65479
[600]	train-auc:0.67176	valid-auc:0.65479
[700]	train-auc:0.67

### 🐱 Entraînement CatBoost

CatBoost excelle avec les features catégorielles et offre une robustesse supplémentaire.

In [36]:
oof_cat, test_cat, score_cat = train_with_cv(X, y, X_test, cat_params, model_type='cat', n_folds=N_FOLDS)


🚀 Entraînement CAT avec 5 folds


────────────────────────────────────────────────────────────
📁 Fold 1/5
────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────
📁 Fold 1/5
────────────────────────────────────────────────────────────
  Train size: 14,127,866 | Val size: 3,531,967
  Train size: 14,127,866 | Val size: 3,531,967
0:	test: 0.6020299	best: 0.6020299 (0)	total: 1.6s	remaining: 26m 42s
0:	test: 0.6020299	best: 0.6020299 (0)	total: 1.6s	remaining: 26m 42s
100:	test: 0.6335166	best: 0.6335166 (100)	total: 2m 8s	remaining: 19m 4s
100:	test: 0.6335166	best: 0.6335166 (100)	total: 2m 8s	remaining: 19m 4s
200:	test: 0.6409180	best: 0.6409180 (200)	total: 4m 16s	remaining: 16m 59s
200:	test: 0.6409180	best: 0.6409180 (200)	total: 4m 16s	remaining: 16m 59s
300:	test: 0.6445025	best: 0.6445025 (300)	total: 6m 34s	remaining: 15m 17s
300:	test: 0.6445025	best: 0.6445025 (300)	total: 6m 34s	remaining: 15m 17s
400:	test: 0

---
## 🎯 8. Ensemble de Modèles - Optimisation des Poids

Au lieu d'une simple moyenne, nous optimisons les poids de chaque modèle pour maximiser l'AUC.  
Cette approche permet de donner plus de poids au(x) meilleur(s) modèle(s).

In [ ]:
from scipy.optimize import minimize

def ensemble_score(weights, *args):
    """
    Fonction objectif pour optimiser les poids de l'ensemble.
    On minimise le négatif de l'AUC (car minimize cherche un minimum).
    """
    oof_preds, y_true = args
    final_pred = np.zeros(len(y_true))
    for i, oof in enumerate(oof_preds):
        final_pred += weights[i] * oof
    return -roc_auc_score(y_true, final_pred)

print("🔍 Optimisation des poids de l'ensemble...\n")

# Optimisation des poids
oof_preds = [oof_lgb, oof_xgb, oof_cat]
initial_weights = [1/3, 1/3, 1/3]  # Poids égaux au départ
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})  # Les poids doivent sommer à 1
bounds = [(0, 1)] * len(oof_preds)  # Chaque poids entre 0 et 1

result = minimize(
    ensemble_score,
    initial_weights,
    args=(oof_preds, y),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

optimal_weights = result.x
print(f"🎯 Poids optimaux trouvés:")
print(f"  LightGBM: {optimal_weights[0]:.4f}")
print(f"  XGBoost:  {optimal_weights[1]:.4f}")
print(f"  CatBoost: {optimal_weights[2]:.4f}")
print(f"  Somme:    {optimal_weights.sum():.4f}\n")

# Prédictions ensemble avec poids optimaux
oof_ensemble = (optimal_weights[0] * oof_lgb + 
                optimal_weights[1] * oof_xgb + 
                optimal_weights[2] * oof_cat)

test_ensemble = (optimal_weights[0] * test_lgb + 
                 optimal_weights[1] * test_xgb + 
                 optimal_weights[2] * test_cat)

score_ensemble = roc_auc_score(y, oof_ensemble)

print(f"{'='*60}")
print(f"📊 COMPARAISON DES SCORES (OOF):")
print(f"{'='*60}")
print(f"  LightGBM:       {score_lgb:.6f}")
print(f"  XGBoost:        {score_xgb:.6f}")
print(f"  CatBoost:       {score_cat:.6f}")
print(f"  {'─'*56}")
print(f"  🏆 ENSEMBLE:     {score_ensemble:.6f}")
print(f"{'='*60}")

# Amélioration par rapport au meilleur modèle individuel
best_individual = max(score_lgb, score_xgb, score_cat)
improvement = (score_ensemble - best_individual) * 100
print(f"\n💡 Amélioration vs meilleur individuel: +{improvement:.4f} points")

---
## 📈 9. Analyse des Prédictions

Visualisation des performances et distribution des prédictions.

Compare les prédictions pour les défauts vs non-défauts

### Courbe ROC

In [ ]:
fpr, tpr, thresholds = roc_curve(y, oof_ensemble)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'Ensemble (AUC = {score_ensemble:.4f})', linewidth=2, color='#2E86AB')
plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.5000)', linewidth=1)
plt.fill_between(fpr, tpr, alpha=0.2, color='#2E86AB')
plt.xlabel('Taux de Faux Positifs (FPR)', fontsize=12)
plt.ylabel('Taux de Vrais Positifs (TPR)', fontsize=12)
plt.title('Courbe ROC - Modèle Ensemble', fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='lower right')
plt.grid(alpha=0.3, linestyle=':')
plt.tight_layout()
plt.show()

print(f"\n📈 Statistiques des prédictions OOF:")
print(pd.Series(oof_ensemble).describe())

### Distribution des prédictions par classe

Visualisons comment le modèle sépare les cas de défaut des non-défauts.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme de densité
axes[0].hist(oof_ensemble[y == 0], bins=50, alpha=0.7, label='Non-défaut (0)', 
             density=True, color='#06D6A0')
axes[0].hist(oof_ensemble[y == 1], bins=50, alpha=0.7, label='Défaut (1)', 
             density=True, color='#EF476F')
axes[0].set_xlabel('Probabilité prédite', fontsize=12)
axes[0].set_ylabel('Densité', fontsize=12)
axes[0].set_title('Distribution des prédictions par classe', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(alpha=0.3, linestyle=':')

# Boxplot comparatif
bp = axes[1].boxplot([oof_ensemble[y == 0], oof_ensemble[y == 1]], 
                      labels=['Non-défaut (0)', 'Défaut (1)'],
                      patch_artist=True,
                      medianprops=dict(color='red', linewidth=2))
bp['boxes'][0].set_facecolor('#06D6A0')
bp['boxes'][1].set_facecolor('#EF476F')
axes[1].set_ylabel('Probabilité prédite', fontsize=12)
axes[1].set_title('Boxplot des prédictions par classe', fontsize=13, fontweight='bold')
axes[1].grid(alpha=0.3, linestyle=':', axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Séparabilité des classes:")
print(f"  Médiane non-défaut: {np.median(oof_ensemble[y==0]):.4f}")
print(f"  Médiane défaut:     {np.median(oof_ensemble[y==1]):.4f}")
print(f"  Écart médian:       {np.median(oof_ensemble[y==1]) - np.median(oof_ensemble[y==0]):.4f}")

---
## 💾 10. Génération des Soumissions

Création des fichiers de soumission au format attendu par la compétition.

In [ ]:
# Création du DataFrame de soumission principale (ensemble)
submission = pd.DataFrame({
    'id': test_id,
    'target': test_ensemble
})

print("\n📄 Aperçu de la soumission ENSEMBLE:")
print(submission.head(10))
print(f"\n📊 Informations:")
print(f"  Shape: {submission.shape}")
print(f"  Colonnes: {submission.columns.tolist()}")
print(f"\n📈 Statistiques des prédictions:")
print(submission['target'].describe())

# Vérifications de sécurité
print(f"\n✅ Vérifications:")
try:
    assert submission['id'].nunique() == len(submission), "IDs dupliqués détectés!"
    print("  ✓ Pas d'IDs dupliqués")
    
    assert submission['target'].between(0, 1).all(), "Probabilités hors de [0,1]!"
    print("  ✓ Toutes les probabilités sont dans [0,1]")
    
    assert not submission.isnull().any().any(), "Valeurs manquantes détectées!"
    print("  ✓ Aucune valeur manquante")
    
    print("\n🎉 Toutes les vérifications passées avec succès!")
except AssertionError as e:
    print(f"\n⚠️ ERREUR: {e}")

### Sauvegarde des fichiers

Nous sauvegardons :
- Le fichier **ensemble** (recommandé pour la soumission)
- Les 3 fichiers individuels (pour comparaison sur le leaderboard)

In [ ]:
# Sauvegarde LightGBM (PRINCIPAL)
submission = pd.DataFrame({'id': test_id, 'target': test_lgb})
submission.to_parquet('submission_lgb.parquet', index=False)
print("✅ Fichier de soumission sauvegardé: submission_lgb.parquet")

print("\n" + "="*60)
print("🎯 SOUMISSION LIGHTGBM:")
print("="*60)
print(f"• AUC validation (OOF): {score_lgb:.6f}")
print(f"• Statistiques des prédictions test:")
print(submission['target'].describe())
print("="*60)

✅ Fichier de soumission sauvegardé: submission_lgb.parquet

🎯 SOUMISSION LIGHTGBM:
• AUC validation (OOF): 0.654171
• Statistiques des prédictions test:
count    654068.000000
mean          0.450041
std           0.119405
min           0.059015
25%           0.372978
50%           0.437676
75%           0.517755
max           0.937326
Name: target, dtype: float64


---
## 📋 11. Points clés

C'est un problème de classification binaire (défaut/non-défaut)

On utilise trois modèles différents pour plus de robustesse

Le feature engineering est crucial pour la performance

La validation croisée assure des résultats fiables

L'ensemble optimisé donne les meilleures performances

In [28]:
# Cellule: afficher score XGBoost, comparaison rapide et sauvegarde optionnelle
print('\n🔎 Résumé XGBoost vs LightGBM')

reported_lgb_score = 0.65417  # score LightGBM rapporté précédemment

try:
    print(f"score_xgb (OOF) = {score_xgb:.6f}")
except Exception as e:
    print(f"score_xgb non défini: {e}")

if 'score_xgb' in globals():
    delta = score_xgb - reported_lgb_score
    print(f"\nΔ AUC (XGB - reported LGB 0.65417): {delta:.6f}")
    if delta > 0:
        print("✅ XGBoost est légèrement meilleur que le LightGBM rapporté.")
    elif delta < 0:
        print("⚠️ LightGBM rapporté est meilleur que XGBoost.")
    else:
        print("⚖️ Scores identiques à la précision affichée.")

    # Sauvegarde optionnelle
    try:
        submission_xgb = pd.DataFrame({'id': test_id, 'target': test_xgb})
        submission_xgb.to_parquet('submission_xgb.parquet', index=False)
        print("\n💾 Fichier sauvegardé: submission_xgb.parquet")
    except Exception as e:
        print(f"\n⚠️ Impossible de sauvegarder la soumission XGB: {e}")
else:
    print('\nℹ️ Exécutez d\'abord la cellule d\'entraînement XGBoost pour définir score_xgb, oof_xgb et test_xgb.')



🔎 Résumé XGBoost vs LightGBM
score_xgb (OOF) = 0.654879

Δ AUC (XGB - reported LGB 0.65417): 0.000709
✅ XGBoost est légèrement meilleur que le LightGBM rapporté.

💾 Fichier sauvegardé: submission_xgb.parquet

💾 Fichier sauvegardé: submission_xgb.parquet
